<a href="https://colab.research.google.com/github/ummd/ummd.github.io/blob/master/Teaching/Lecture5/Lecture5c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robust Estimator of Covariance Matrix

## A dataset without heteroskedasticity

In [ ]:
if (!require(lmtest)) install.packages("lmtest")
if (!require(sandwich)) install.packages("sandwich")
library(lmtest)
library(sandwich)


Loading required package: lmertest

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lmertest’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘lmertest’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message:
“Perhaps you meant ‘lmerTest’ ?”


ERROR: ignored

In [ ]:

set.seed(1)
N<-50
# generate linear regression relationship
# with Homoskedastic variances
b<-.1

x <- sin(1:N)
y <- 1 + b*x + 5*rnorm(N)
## model fit
lm.fit <- lm(y ~ x)

summary(lm.fit)


Call:
lm(formula = y ~ x)

Residuals:
     Min       1Q   Median       3Q      Max 
-11.2556  -2.5615   0.1541   3.0105   7.2307 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)   1.5016     0.5931   2.532   0.0147 *
x            -0.2225     0.8368  -0.266   0.7914  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.194 on 48 degrees of freedom
Multiple R-squared:  0.001471,	Adjusted R-squared:  -0.01933 
F-statistic: 0.07073 on 1 and 48 DF,  p-value: 0.7914


## With heteroskdasticity

In [ ]:
set.seed(1)

y <- 1 + b*x + 5*rnorm(N,mean=0,sd=1+x) #now the sd is a function of x, so it depends on the value of x for each observation

We can fit the linear model, but the standard error will be incorrect because it assumes that the variance of the observations is constant

In [ ]:

lm.fit <- lm(y ~ x)
summary(lm.fit)


Call:
lm(formula = y ~ x)

Residuals:
     Min       1Q   Median       3Q      Max 
-22.5372  -1.0318  -0.1718   2.8359  10.3111 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)   1.3395     0.7375   1.816   0.0756 .
x             0.2561     1.0406   0.246   0.8066  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.215 on 48 degrees of freedom
Multiple R-squared:  0.00126,	Adjusted R-squared:  -0.01955 
F-statistic: 0.06057 on 1 and 48 DF,  p-value: 0.8066


The equation used by lm reports a standard error of:

In [ ]:
summary(lm.fit)$coef[2,2]

[1] 1.040575

But if we simulate a large number of samples, the actual variance of the coefficient is bigger

In [ ]:
S=1000
e <- replicate(S,rnorm(N,mean=0,sd=1+x))
y <- 1 + b*x + 5*e

bh=matrix(,2,S) #initialize matrix to store estimates of beta

for (s in 1:S){
    lm.fit.sims=lm(y[,s]~x) # estimate the linear model on the sample s.
    bh[1:2,s] <- coef(lm.fit.sims)   #store the estimates
}

bh0=bh[1,]
bh1=bh[2,]

sd(bh1)

[1] 1.322888

### Test with "robust" standard errors
Using the coeftest() function, we can calculate a "robust" standard error. Note that it is larger then the one calcuclated from the (incorrect) linear model which assumes that the variance of each observation is equal.

In [ ]:
coeftest(lm.fit, vcov. = vcovHC)


t test of coefficients:

            Estimate Std. Error t value Pr(>|t|)  
(Intercept)  1.33954    0.76165  1.7587   0.0850 .
x            0.25609    1.27954  0.2001   0.8422  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Heterogeneity - Clustering

The simple linear model has an important assumption, the independence of the observations. This assumption holds in most carefully designed experiments but seldom does in real-life datasets.

One of the biggest risks of assuming correlated data as independent is that your linear model will always give you a beautiful p-value due to a large number of “independent” observations.

The “correlation” of observations usually comes from some shared features by the data points within the same group. For example, if you are interested in how family income affects children’s exam grades, you need to consider that the students’ grades from the same school or class are more similar to each other than those from different schools or classes.

In [ ]:
set.seed(1)
N<-3
S<-50

# generate linear regression relationship
b1 <- 1
x <- runif(S*N)

b0.s <- rep(rnorm(1),N) # subject-specific effect
G <- rep('1',N) # subject label
for (s in 2:S){
  b0.s <- c(b0.s,rep(rnorm(1),N))
  G <- c(G,rep(as.character(s),N))
}

#the outcome is a combination of the subject-specific effect and the main effect of x
y <-  b0.s+ b1*x + 1.6*rnorm(S*N)


In [ ]:
## model fit
lm.fit <- lm(y ~ x)
summary(lm.fit)


Call:
lm(formula = y ~ x)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.1865 -1.1535 -0.1014  1.1067  4.5673 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)  -0.1055     0.3327  -0.317   0.7517  
x             1.2625     0.5751   2.195   0.0297 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.897 on 148 degrees of freedom
Multiple R-squared:  0.03153,	Adjusted R-squared:  0.02499 
F-statistic: 4.819 on 1 and 148 DF,  p-value: 0.02971


In [ ]:
coeftest(lm.fit, vcov. = vcovCL, cluster = ~ G)


t test of coefficients:

            Estimate Std. Error t value Pr(>|t|)  
(Intercept) -0.10546    0.38909 -0.2710  0.78675  
x            1.26247    0.66071  1.9108  0.05797 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [ ]:
## model fit
lm.fit <- lm(y ~ b0.s+x)
summary(lm.fit)


Call:
lm(formula = y ~ SS + x)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.4245 -0.9800 -0.1023  1.0515  4.3417 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.0882     0.2903  -0.304   0.7617    
SS            1.1782     0.1712   6.883 1.58e-10 ***
x             1.0285     0.5030   2.045   0.0427 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.655 on 147 degrees of freedom
Multiple R-squared:  0.2676,	Adjusted R-squared:  0.2576 
F-statistic: 26.85 on 2 and 147 DF,  p-value: 1.15e-10


In [ ]:
lm.fit <- lm(y ~ G+x)
summary(lm.fit)


Call:
lm(formula = y ~ G + x)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5886 -0.9110 -0.0035  0.8264  4.0538 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept)  0.620036   1.024621   0.605   0.5465  
G10         -1.474428   1.401402  -1.052   0.2953  
G11         -0.648356   1.401178  -0.463   0.6446  
G12          0.939227   1.402732   0.670   0.5047  
G13         -0.003729   1.401866  -0.003   0.9979  
G14          0.472414   1.406635   0.336   0.7377  
G15         -1.066961   1.406323  -0.759   0.4498  
G16          0.376846   1.398971   0.269   0.7882  
G17          0.362301   1.407186   0.257   0.7974  
G18          2.625140   1.400800   1.874   0.0639 .
G19          0.595571   1.407969   0.423   0.6732  
G2          -1.554786   1.409910  -1.103   0.2728  
G20          0.589675   1.401336   0.421   0.6748  
G21          0.237059   1.402466   0.169   0.8661  
G22         -2.706449   1.398878  -1.935   0.0559 .
G23         -0.917723   1.39

## Random Effects
In such cases, using random effects is an efficient way to improve the estimates in the linear models. Generally speaking, if you have some grouping structures in the investigating dataset that are not directly related to your keen question to answer, it’s better to include them as random effects in your linear model.

In [ ]:

if (!require(lmerTest)) install.packages("lmerTest")
if (!require(lme4)) install.packages("lme4")
library(lme4)
library(lmerTest)

In [ ]:
lm.fit <- lmer(y ~ (1|G)+x)
summary(lm.fit)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: y ~ (1 | G) + x

REML criterion at convergence: 612.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3239 -0.5860 -0.1205  0.5820  2.3715 

Random effects:
 Groups   Name        Variance Std.Dev.
 G        (Intercept) 0.6773   0.823   
 Residual             2.9299   1.712   
Number of obs: 150, groups:  G, 50

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)  
(Intercept)  -0.06233    0.34185 134.76982  -0.182   0.8556  
x             1.17822    0.56546 145.68987   2.084   0.0389 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
  (Intr)
x -0.847

### What if the main effect also varies across subject?

In [ ]:
set.seed(1)
N<-4
S<-50

# generate linear regression relationship
x <- runif(S*N)

b1.s <- rep(rnorm(1,mean=1,sd=1),N) #the size of main effect is a draw from the normal(1,1)
b0.s <- rep(rnorm(1),N) # subject-specific effect
G <- rep('1',N) # subject label
for (s in 2:S){
  b1.s <- c(b1.s,rep(rnorm(1,mean=1,sd=1),N))
  b0.s <- c(b0.s,rep(rnorm(1),N))
  G <- c(G,rep(as.character(s),N))
}

#the outcome is a combination of the subject-specific effect and the main effect of x, however the size of the main effect of x varies randomly across subjects
y <-  b0.s+ b1.s*x + 1.6*rnorm(S*N)

In [ ]:
lm.fit <- lm(y ~ x)
summary(lm.fit)


Call:
lm(formula = y ~ x)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.1171 -1.2115 -0.0724  1.3731  4.9892 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept)   0.1445     0.2923   0.494    0.622
x             0.7209     0.5013   1.438    0.152

Residual standard error: 1.903 on 198 degrees of freedom
Multiple R-squared:  0.01034,	Adjusted R-squared:  0.005337 
F-statistic: 2.068 on 1 and 198 DF,  p-value: 0.152


In [ ]:
lm.fit <- lmer(y ~ x+(1+x|G))
summary(lm.fit)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: y ~ x + (1 + x | G)

REML criterion at convergence: 807.9

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.45778 -0.65526 -0.02213  0.65941  2.04243 

Random effects:
 Groups   Name        Variance Std.Dev. Corr
 G        (Intercept) 0.2663   0.516        
          x           1.3756   1.173    0.40
 Residual             2.6609   1.631        
Number of obs: 200, groups:  G, 50

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)  
(Intercept)  0.05972    0.27476 29.56171   0.217   0.8294  
x            0.89821    0.49901 43.85554   1.800   0.0787 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
  (Intr)
x -0.781